In [17]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns

pred_a = pd.read_csv('pred_A_gb_3_absolute_error.csv')
pred_a['date'] = pd.to_datetime(pred_a['date_forecast_dt'])
pred_b = pd.read_csv('pred_B_gb_3_absolute_error.csv')
pred_b['date'] = pd.to_datetime(pred_b['date_forecast_dt'])
pred_c = pd.read_csv('pred_C_gb_3_absolute_error.csv')
pred_c['date'] = pd.to_datetime(pred_c['date_forecast_dt'])

test = pd.read_csv('data/test.csv')
test['time'] = pd.to_datetime(test['time'])

In [18]:
submission = pd.DataFrame(columns=['prediction'])

for val in pred_a['date']:
    if test['time'].eq(val).any():
        row = pred_a.loc[pred_a['date'] == val]
        submission = submission.append({'prediction': float(row['prediction'])}, ignore_index=True)

for val in pred_b['date']:
    if test['time'].eq(val).any():
        row = pred_b.loc[pred_a['date'] == val]
        submission = submission.append({'prediction': float(row['prediction'])}, ignore_index=True)


for val in pred_c['date']:
    if test['time'].eq(val).any():
        row = pred_c.loc[pred_a['date'] == val]
        submission = submission.append({'prediction': float(row['prediction'])}, ignore_index=True)
        
        
submission.to_csv('submission_gb_absolute_error_2.csv')

In [14]:
def calcGrade(VT_defeated=5):
    number_score = 41 + (100-41)/(10-1) * (VT_defeated-1)
    print(number_score)
    
calcGrade()

67.22222222222223


In [31]:
import utils

In [32]:
train_a, test_a = utils.preprocess_category('A')


Total data points: 34085
Data points to be removed: 0


In [33]:
train_a_Y = train_a['pv_measurement']
train_a_X = train_a.drop(columns=['pv_measurement'])
train_a_X = train_a_X.drop(columns=['date_calc', 'date_forecast', 'time'])

In [40]:
import lightgbm as lgb
import optuna
from sklearn.model_selection import cross_val_score

In [35]:
def clean_feature_name(name):
    # Replace any special characters you want here
    return name.replace(':', '_').replace('{', '_').replace('}', '_').replace('"', '_')

train_a_X.columns = [clean_feature_name(col) for col in train_a_X.columns]
print(train_a_X)


        absolute_humidity_2m_gm3  air_density_2m_kgm3  ceiling_height_agl_m  \
0                            7.7                1.230           1744.900024   
1                            7.7                1.229           1734.000000   
2                            7.7                1.228           1723.500000   
3                            7.7                1.226           1713.400024   
4                            7.7                1.225           1703.599976   
...                          ...                  ...                   ...   
136237                       4.5                1.281           1532.900024   
136238                       4.5                1.281           1413.199951   
136239                       4.5                1.281           1293.599976   
136240                       4.5                1.281           1173.900024   
136241                       4.5                1.281           1054.199951   

        clear_sky_energy_1h_J  clear_sky_rad_W  clo

In [41]:
def objective(trial):
    dtrain = lgb.Dataset(train_a_X, label=train_a_Y)
    param = {
        "objective": "regression",
        "metric": "rmse",
        "verbosity": -1,
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.1, step=0.01),
        "boosting_type": trial.suggest_categorical("boosting_type", ['dart', 'gbdt']),
        "lambda_l1": trial.suggest_float("lambda_l1", 1e-8, 10.0, log=True),
        "lambda_l2": trial.suggest_float("lambda_l2", 1e-8, 10.0, log=True),
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.4, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.4, 1.0),
        "bagging_freq": trial.suggest_int("bagging_freq", 1, 7),
        "min_child_samples": trial.suggest_int("min_child_samples", 5, 100),
        "verbose": -1
    }

    model = lgb.LGBMRegressor(**param)
    model.fit(train_a_X, train_a_Y)

#     result = cross_val_score(model, train_X_std.values, train_Y.values.ravel(), cv=5, scoring="neg_mean_squared_log_error")
    result = cross_val_score(model, train_a_X.values, train_a_Y.values.ravel(), cv=5, scoring="neg_mean_squared_error")


    tot = 0
    for part in result:
        tot = tot + (-part)
    accuracy = tot/len(result)

    return accuracy


In [42]:

study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

[I 2023-10-31 15:02:52,604] A new study created in memory with name: no-name-21c6be65-4c3a-4edc-99f8-0a70a2d5b42e


[LightGBM] [Warning] feature_fraction is set=0.8533549082272982, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8533549082272982
[LightGBM] [Warning] lambda_l1 is set=0.00032292141549206555, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.00032292141549206555
[LightGBM] [Warning] bagging_fraction is set=0.7559834467811313, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7559834467811313
[LightGBM] [Warning] lambda_l2 is set=1.6716148362149374, reg_lambda=0.0 will be ignored. Current value: lambda_l2=1.6716148362149374
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
[LightGBM] [Warning] feature_fraction is set=0.8533549082272982, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8533549082272982
[LightGBM] [Warning] lambda_l1 is set=0.00032292141549206555, re

[I 2023-10-31 15:03:08,470] Trial 0 finished with value: 353716.1001310247 and parameters: {'learning_rate': 0.03, 'boosting_type': 'dart', 'lambda_l1': 0.00032292141549206555, 'lambda_l2': 1.6716148362149374, 'num_leaves': 136, 'feature_fraction': 0.8533549082272982, 'bagging_fraction': 0.7559834467811313, 'bagging_freq': 2, 'min_child_samples': 74}. Best is trial 0 with value: 353716.1001310247.


[LightGBM] [Warning] feature_fraction is set=0.8138697061175596, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8138697061175596
[LightGBM] [Warning] lambda_l1 is set=0.0009512956001389622, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0009512956001389622
[LightGBM] [Warning] bagging_fraction is set=0.669871611627519, subsample=1.0 will be ignored. Current value: bagging_fraction=0.669871611627519
[LightGBM] [Warning] lambda_l2 is set=0.6582688070139822, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.6582688070139822
[LightGBM] [Warning] bagging_freq is set=2, subsample_freq=0 will be ignored. Current value: bagging_freq=2
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
[LightGBM] [Warning] feature_fraction is set=0.8138697061175596, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8138697061175596
[LightGBM] [Warning] lambda_l1 is set=0.0009512956001389622, reg_alp

[I 2023-10-31 15:03:21,067] Trial 1 finished with value: 350508.405830701 and parameters: {'learning_rate': 0.01, 'boosting_type': 'gbdt', 'lambda_l1': 0.0009512956001389622, 'lambda_l2': 0.6582688070139822, 'num_leaves': 115, 'feature_fraction': 0.8138697061175596, 'bagging_fraction': 0.669871611627519, 'bagging_freq': 2, 'min_child_samples': 37}. Best is trial 1 with value: 350508.405830701.


...

[LightGBM] [Warning] feature_fraction is set=0.4853425837488361, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4853425837488361
[LightGBM] [Warning] lambda_l1 is set=2.6183474401875785e-05, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.6183474401875785e-05
[LightGBM] [Warning] bagging_fraction is set=0.6121375430101864, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6121375430101864
[LightGBM] [Warning] lambda_l2 is set=4.250764179521765e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=4.250764179521765e-05
[LightGBM] [Warning] bagging_freq is set=7, subsample_freq=0 will be ignored. Current value: bagging_freq=7
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
[LightGBM] [Warning] feature_fraction is set=0.4853425837488361, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4853425837488361
[LightGBM] [Warning] lambda_l1 is set=2.6183474401875785e-

[I 2023-10-31 15:18:12,427] Trial 98 finished with value: 188239.18525874586 and parameters: {'learning_rate': 0.060000000000000005, 'boosting_type': 'gbdt', 'lambda_l1': 2.6183474401875785e-05, 'lambda_l2': 4.250764179521765e-05, 'num_leaves': 29, 'feature_fraction': 0.4853425837488361, 'bagging_fraction': 0.6121375430101864, 'bagging_freq': 7, 'min_child_samples': 40}. Best is trial 52 with value: 187964.2547516551.


[LightGBM] [Warning] feature_fraction is set=0.4373261707540464, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4373261707540464
[LightGBM] [Warning] lambda_l1 is set=2.546490807935427e-06, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.546490807935427e-06
[LightGBM] [Warning] bagging_fraction is set=0.6142183111584019, subsample=1.0 will be ignored. Current value: bagging_fraction=0.6142183111584019
[LightGBM] [Warning] lambda_l2 is set=3.639376720495259e-05, reg_lambda=0.0 will be ignored. Current value: lambda_l2=3.639376720495259e-05
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] verbosity is set=-1, verbose=-1 will be ignored. Current value: verbosity=-1
[LightGBM] [Warning] feature_fraction is set=0.4373261707540464, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.4373261707540464
[LightGBM] [Warning] lambda_l1 is set=2.546490807935427e-06,

[I 2023-10-31 15:18:22,485] Trial 99 finished with value: 190616.52029564395 and parameters: {'learning_rate': 0.060000000000000005, 'boosting_type': 'gbdt', 'lambda_l1': 2.546490807935427e-06, 'lambda_l2': 3.639376720495259e-05, 'num_leaves': 97, 'feature_fraction': 0.4373261707540464, 'bagging_fraction': 0.6142183111584019, 'bagging_freq': 5, 'min_child_samples': 39}. Best is trial 52 with value: 187964.2547516551.


Number of finished trials: 100
Best trial:
  Value: 187964.2547516551
  Params: 
    learning_rate: 0.060000000000000005
    boosting_type: gbdt
    lambda_l1: 2.824103268615774
    lambda_l2: 0.0009800624138226444
    num_leaves: 33
    feature_fraction: 0.4704407678025172
    bagging_fraction: 0.7871936139792188
    bagging_freq: 7
    min_child_samples: 77
